In [10]:
# jupyter server url : http://127.0.0.1:8889/?token=feee72ecb0de9269728ba7e915647a4e8f50dd477058e9e1

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("PierreMihEval2") \
    .config("spark.ui.port","4040")\
    .config("spark.executor.memory","10g")\
    .config("spark.driver.memory","4g")\
    .config("spark.jars", "/opt/spark/jars/mysql-connector-j-9.0.0.jar")\
    .getOrCreate()

In [12]:
user = "thegameadministrator"
password = "thegameadministratorpassword"
driver = "com.mysql.jdbc.Driver"
schemaName = "thegame"
properties = {"user":user,
              "driver":driver,
              "password":password
              }
connectionString = "jdbc:mysql://db"
players_table_name = "players"
player_matches_table_name = "player_matches"
player_message_sentiment_table_name = "player_messages"
player_transactions_table_name = "player_transactions"
transaction_products_table_name = "transaction_products"

In [13]:
df_players = spark.read.jdbc(connectionString, schemaName + "." + players_table_name, properties=properties)
df_player_matches = spark.read.jdbc(connectionString, schemaName + "." + player_matches_table_name, properties=properties)
df_player_messages_sentiment = spark.read.jdbc(connectionString, schemaName + "." + player_message_sentiment_table_name, properties=properties)
df_player_transactions = spark.read.jdbc(connectionString, schemaName + "." + player_transactions_table_name, properties=properties)
df_transaction_products = spark.read.jdbc(connectionString, schemaName + "." + transaction_products_table_name, properties=properties)

In [14]:
df_players.show(1)
df_player_matches.show(1)
df_player_messages_sentiment.show(1)
df_player_transactions.show(1)
df_transaction_products.show(1)

+----+-----+------------------+
|  id|level|total_hours_played|
+----+-----+------------------+
|4761|   13|               390|
+----+-----+------------------+
only showing top 1 row

+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
|player_id|                date|kills|deaths|friendly_kills|time_in_movement_ration|shots_fired|flags_captured|
+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
|      948|2023-11-28Z03:34:59Z|    0|    15|             1|                   0.76|        508|             8|
+---------+--------------------+-----+------+--------------+-----------------------+-----------+--------------+
only showing top 1 row

+----+--------------------+---------+
|  id|                date|sentiment|
+----+--------------------+---------+
|4761|2021-12-27Z05:57:26Z|     0.54|
+----+--------------------+---------+
only showing top 1 row

+-------+---------

In [ ]:
from pyspark.sql.functions import col

df_players_plus = df_players.withColumn("hours_per_level", col("total_hours_played") / col("level"))
df_players_plus.select("level", "total_hours_played", "hours_per_level").describe().show()
level_to_check = 5
df_players_plus.where(col("level") > level_to_check).orderBy("total_hours_played", ascending=True).show()